In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2
import yaml
import sys
import os
from IPython import display
import time
from google.colab import drive
import torch
import torch.nn as nn
# from tqdm import tqdm
# from PIL import Image

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if torch.cuda.is_available():

  import torchvision
  print("PyTorch version:", torch.__version__)
  print("Torchvision version:", torchvision.__version__)
  print("CUDA is available:", torch.cuda.is_available())
  import sys
  !{sys.executable} -m pip install opencv-python matplotlib
  !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
  from segment_anything.utils.amg import *
  from pycocotools import mask as mask_utils

PyTorch version: 2.0.1+cu118
Torchvision version: 0.15.2+cu118
CUDA is available: True
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-ey53hf0a
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-ey53hf0a
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done


In [ ]:
categories = [
      'Metal (general)',
      'Metal (iron bender)',
      'Metal (pipe)',
      'Wood (pallett)',
      'Wood (scraps/cuttings)',
      'Carton',
      'Concrete',
      'Plastic (big bag)',
      'Plastic (bucket)',
      'Plastic (general)',
      'Plastic (pipe)',
      'Plastic (sand bag)',
      'Nylon',
      'Rubber',
      'Textil/Fabric/Cloth',
      'Background',
      'All Background',
      'Unknown',
      'Skip',
      'Back',
      'Exit'
  ]

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    polygons = []
    color = []
    for ann in sorted_anns:
        m = rle_to_mask(ann['segmentation'])
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
        ax.imshow(np.dstack((img, m*0.35)))

In [ ]:
def get_box_from_mask(mask):
  mask = rle_to_mask(mask)
  pixel_list = np.argwhere(mask)
  min_x = min(pixel[1] for pixel in pixel_list)
  min_y = min(pixel[0] for pixel in pixel_list)
  max_x = max(pixel[1] for pixel in pixel_list)
  max_y = max(pixel[0] for pixel in pixel_list)
  box = [min_x, min_y, max_x, max_y]
  return box

In [ ]:
# def crop_object(image, mask):
#     cropped = []
#     for i in range(mask.shape[0]):
#         img = image[i,:,:,:].squeeze()
#         m = mask[i,:,:].squeeze()
#         contours, _ = cv2.findContours(m.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#         largest_contour = max(contours, key=cv2.contourArea)
#         contour_mask = np.zeros_like(m, dtype=np.uint8)
#         cv2.drawContours(contour_mask, [largest_contour], 0, 255, -1)
#         cropped_image = cv2.bitwise_and(img, img, mask=contour_mask)
#         cropped.append(cropped_image)
#     return np.array(cropped)

In [ ]:
import torch.nn.functional as F
from skimage import transform, img_as_ubyte

class base(nn.Module):
  def __init__(self):
    super(base, self).__init__()
    self.fc1 = nn.Linear(3, 64)
    self.fc2 = nn.Linear(64, 64)
    self.fc3 = nn.Linear(64, 9)
    self.seq = nn.Sequential(self.fc1, nn.ReLU(), self.fc2, nn.ReLU(), self.fc3)

  def forward(self, x):
    image, mask = x
    masked = (mask.unsqueeze(3) * image).cuda()
    col = masked.sum(axis=[1, 2]) / mask.sum()
    x = self.seq(col)
    return x

In [ ]:
folder_path = "/content/drive/MyDrive/Project/Project/data/Arik"

In [ ]:
image = 'DJI_20230221091035_0034_V'
split = 0

In [ ]:
labels_dict = {'Metal (general)':0, 'Metal (iron bender)':0, 'Metal (pipe)':0,
               'Wood (pallett)':1, 'Wood (scraps/cuttings)':1,
               'Carton':2,
               'Concrete':3,
               'Plastic (big bag)':4, 'Plastic (bucket)':4, 'Plastic (general)':4,
               'Plastic (pipe)':4, 'Plastic (sand bag)':4,
               'Nylon':5,
               'Rubber':6,
               'Textil/Fabric/Cloth':7,
               'Background':8,
               'Unknown':9}

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

class SplitDataset(Dataset):
  def __init__(self, folder_path, image_name, sp):
    self.image_name = image_name
    self.sp = sp
    self.im_path = f'{folder_path}/{image_name}'
    self.split_path = f'{self.im_path}/split_{sp}'
    with open(f'{self.split_path}/seg_{sp}.yaml') as f:
      self.seg = yaml.safe_load(f)
    files_in_folder = os.listdir(self.split_path)
    self.ann_dict={image_name:{sp:{}}}
    if f'anns_{sp}.yaml' in files_in_folder:
      with open(f'{self.split_path}/anns_{sp}.yaml', 'r') as f:
        self.ann_dict = yaml.safe_load(f)
        # Remove all unknowns
        self.ann_dict[self.image_name][self.sp] = {key:val for key, val in self.ann_dict[self.image_name][self.sp].items() if val != 'Unknown'}

  def __len__(self):
    return len(self.ann_dict[self.image_name][self.sp])

  def __getitem__(self, idx):
    annotated_masks = list(self.ann_dict[self.image_name][self.sp].keys())
    m = annotated_masks[idx]
    img = cv2.imread(f'{self.split_path}/split_{self.sp}.jpg')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = rle_to_mask(self.seg[self.image_name][self.sp][m]['segmentation'])
    label = self.ann_dict[self.image_name][self.sp][m]
    y = labels_dict[label]
    return (img, mask), y

In [ ]:
arr = []
for split in range(16):
  ds = SplitDataset(folder_path, image, split)
  arr.append(ds)

curr_dataset = torch.utils.data.ConcatDataset(arr)

In [ ]:
print(curr_dataset[0])

In [ ]:
# curr_dataset = []
# main_data_folder = "/content/drive/MyDrive/Project/Project/data"
# for name in os.listdir(main_data_folder):
#   for image in os.listdir(f'{main_data_folder}/{name}'):
#     for split in range(16):
#       ds = SplitDataset(f'{main_data_folder}/{name}', image, split)
#       curr_dataset = torch.utils.data.ConcatDataset([curr_dataset, ds])

In [ ]:
train_ds, val_ds, test_ds = torch.utils.data.random_split(curr_dataset, [0.6, 0.2, 0.2])
train_dataloader = DataLoader(train_ds, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(val_ds, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=32, shuffle=True)


In [ ]:
# Model parameters
model = base().cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = torch.nn.CrossEntropyLoss().cuda()

In [ ]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.
    running_accuracy = 0.
    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        labels = labels.cuda()

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs).cuda()
        # outputs = model(inputs)
        running_accuracy += (labels == outputs.argmax(dim=1)).float().mean()
        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 10 == 9:
            last_loss = running_loss / 10 # loss per batch
            last_accuracy = running_accuracy / 10
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            print('  batch {} accuracy: {}'.format(i + 1, last_accuracy))
            tb_x = epoch_index * len(train_dataloader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss, last_accuracy

In [ ]:
from datetime import datetime
from torch.utils.tensorboard.writer import SummaryWriter

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 20

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss, avg_accuracy = train_one_epoch(epoch_number, writer)


    running_vloss = 0.0
    running_vaccuracy = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(valid_dataloader):
            vinputs, vlabels = vdata
            vlabels = vlabels.cuda()
            voutputs = model(vinputs).cuda()
            # voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vaccuracy += (vlabels == voutputs.argmax(dim=1)).float().mean()
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    avg_vaccuracy = running_vaccuracy / (i + 1)

    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
    print('ACCURACY train {} valid {}'.format(avg_accuracy, avg_vaccuracy))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:


RuntimeError: ignored